# Tutorial on interacting with ReaktoroBlock
Demonstration of how use Reaktoro block.

## Dependencies
* Python - Programming language
* Pyomo - Python package for equation-oriented modeling
* IDAES - Python package extending Pyomo for flowsheet modeling
* cyipopt - Solver necessary for use with gray box models
* Reaktoro-pse - Python package for building Reaktoro gray box models
* WaterTAP - Unit models

## Demonstration structure 
* Background on speciation and Reaktoro-pse philosophy 
* Setting up basic speciation block and calculating properties for a sea water composition
    * Demonstrate key configuration options
    * Key display options
    * How to adjust apparent species to achieve thermodynamic zero charge
    * Adding chemical to adjust pH


## Reaktoro-PSE API and structure 
Reaktoro-pse is a wrapper for enabling use of [Reaktoro](https://reaktoro.org/) as a [Graybox model](https://pyomo.readthedocs.io/en/stable/contributed_packages/pynumero/pynumero.interfaces.external_grey_box_model.html) in IDAES compatible modeling platforms such as WaterTAP. This is not a replacement for Reaktoro or higher level API for Reaktoro. 

The general objective is to provide a structure that enables user to automatically build Reaktoro Graybox model by specifying:

* Input ion composition (apprant  or exact species)
* System temperature, pressure, pH, and charge neutrality
* Rekatoro databases and activity models
* Outputs supported by Reaktoro and custom outputs built using Reaktoro database information and outputs 

The general API structure is shown in Figure bellow. The figure shows the type of inputs and outputs and how they are handled by core api to configure Reaktoro Graybox model. 

<img src="reaktoro_pse_api.png" width="1000" height="650">

Reaktoro-pse supports all available databases and activity models provided by Reaktoro:
* Please refer here for information on [databases](https://reaktoro.org/tutorials/basics/loading-databases.html) (all are supported) 
* Please refer here for information on [activity models](https://reaktoro.org/tutorials/basics/specifying-activity-models.html) (all are supported, included chain operations, or passing in pre-configured activity models) 

Reaktoro-pse supports all properties that provide as single floating point or real value as an output from chemical and aqueous properties: 
* [Chemical properties](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html)
* [Aqueous properties](https://reaktoro.org/api/classReaktoro_1_1AqueousProps.html)
* Pyomo build properties, which are custom properties built in Pyomo that use chemical properties or aqueous properties 

## ReaktoroBlock builds Reaktoro Graybox usings standard IDAES StateBlock methods

The ReaktoroBlock automates construction of reaktoro Graybox as a IDAES [StateBlock](https://idaes-pse.readthedocs.io/en/stable/explanations/components/property_package/state_block.html), providing range of configuration of options to simplify performing equilibrium chemistry calculations:

* Enables construction of speciation block and propagation of exact spetiation to property block to enable chemistry modification
* Supports indexing 
* Provides options to configure reaktoro property block (and if constructed speciation block) 
* Uses core api functionality to automatically scale input and output variables and constraints if user does not provide any
* Automatically scales gray box jacobian 
* Provides defaults to simplify configuration and usage for typical calculations 

**Minimal configuration options required for RreaktoroBlock:**
* Database and database file selection from Reaktoro
* Activity models for phases being considered from Reaktoro
* Input apparent or exact species mol or mass flows (**concentrations are not supported!**)
* If speciation block is needed (if exact speciation is not provided then generally yes)
* System states (temperature, pressure, or pH, unless they are being solved for)
* Outputs of interest 

### Getting basic properties from ReaktoroBlock
<img src="direct_prop.png" width="800" height="300">

## Import needed modules

In [105]:
## Import core components
# Pyomo core components
from pyomo.environ import (
    Var,
    Constraint,
    ConcreteModel,
    Block,
    assert_optimal_termination,
    units as pyunits,
)
# Ideas core components
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import (
    calculate_scaling_factors,
    set_scaling_factor,
    constraint_scaling_transform,
)

from idaes.core.util.model_statistics import degrees_of_freedom
from watertap.core.solvers import get_solver

from pyomo.util.calc_var_value import calculate_variable_from_constraint

# WaterTAP core components
import watertap.property_models.NaCl_prop_pack as properties

# Import reaktoro-pse and reaktoro
from reaktoro_pse.reaktoro_block import ReaktoroBlock
import reaktoro

Define a feed comopostion, here we use seawater

In [106]:
"""This a typical composition of sea water with ion concentration in mg/L and pH"""

sea_water_composition = {
    "Na": 10556,
    "K": 380,
    "Ca": 400,
    "Mg": 1262,
    "Cl": 17000, # this is lower then tpyical to emulate a feed that was not charge balanced
    "SO4": 2649,
    "HCO3": 140,
}
sea_water_ph = 7.56

### Define standard Pyomo model and WaterTAP property package

In [107]:
m = ConcreteModel()
# create IDAES flowsheet
m.fs = FlowsheetBlock(dynamic=False)


### Define pyomo variables for sea_water state, composition, and properties of interest

In [108]:
''' build block for holding sea water properties'''
m.fs.sea_water=Block()
"""temperature"""
m.fs.sea_water.temperature = Var(
    initialize=293, bounds=(0,300), units=pyunits.K)
m.fs.sea_water.temperature.fix()
set_scaling_factor(m.fs.sea_water.temperature, 1/293)
"""pressure"""
m.fs.sea_water.pressure = Var(
    initialize=1e5, units=pyunits.Pa)
m.fs.sea_water.pressure.fix()
set_scaling_factor(m.fs.sea_water.pressure, 1/1e5)
"""pH"""
m.fs.sea_water.pH = Var(initialize=sea_water_ph)
m.fs.sea_water.pH.fix()
set_scaling_factor(m.fs.sea_water.pH, 1)

Define composition and mass flows. ReaktoroBlock requires mass flows or mol flows as input for composition

In [109]:
"""ion concentration variable"""
ions = list(sea_water_composition.keys())
#
m.fs.sea_water.species_concentrations = Var(
    ions, initialize=1, units=pyunits.mg / pyunits.L
)

""" mass flows of all species, including water"""
ions.append("H2O")
m.fs.sea_water.species_mass_flow = Var(
    ions, initialize=1,  units=pyunits.kg / pyunits.s
)


Create output properties variables

In [110]:

"""Charge neutrality"""
m.fs.sea_water.charge = Var(initialize=0)
set_scaling_factor(m.fs.sea_water.charge, 1e8)

"""Solution density"""
m.fs.sea_water.density = Var(
    initialize=1000, units=pyunits.kg / pyunits.m**3
)  
set_scaling_factor(m.fs.sea_water.density, 1e-3)
"""Osmotic pressure"""
m.fs.sea_water.osmotic_pressure = Var(initialize=1, units=pyunits.Pa)
set_scaling_factor(m.fs.sea_water.osmotic_pressure, 1e-5)


Build constraints for calculation of mass flows

In [111]:
"""Write constraints to convert concentration to mass flows"""
@m.fs.sea_water.Constraint(list(m.fs.sea_water.species_concentrations.keys()))
def eq_sea_water_species_mass_flow(fs, ion):
    """calculate mass flow based on density"""
    return m.fs.sea_water.species_mass_flow[ion] == pyunits.convert(
        m.fs.sea_water.species_concentrations[ion]
        * m.fs.sea_water.species_mass_flow["H2O"]
        / m.fs.sea_water.density,
        to_units=pyunits.kg / pyunits.s,
    )


### Put all outputs into a single dictionary for use in ReaktoroBlock
Note how we defined "speciesAmount":True in dict, this will also return all true species based on provided database and apparent species

In [112]:
"""We need to define an output dictionary with our properties - this can also be an Pyomo IndexedVar please check examples in examples folder for how to use IndexedVars as outputs"""

m.fs.sea_water.outputs = {
    (
        "osmoticPressure",
        "H2O",
    ): m.fs.sea_water.osmotic_pressure,  # not how the second key is the water, we can get osmotic pressure for different components in the system
    ("density", None): m.fs.sea_water.density,
    ("charge", None): m.fs.sea_water.charge,
    "speciesAmount": True, } # - this will force reaktor to return exact speciation with all species

### Configure ReaktoroBlock
We choose PhreeqC Pitzer database is its a good choice for high salinity feeds like sea water. 
For comparison of PhreeqC data base following paper is a good reference: https://doi.org/10.1016/j.earscirev.2021.103888 

In [113]:
m.fs.sea_water.eq_reaktoro_properties = ReaktoroBlock(
    system_state={
        "temperature": m.fs.sea_water.temperature,
        "pressure": m.fs.sea_water.pressure,
        "pH": m.fs.sea_water.pH,
    },
    aqueous_phase={
        "composition": m.fs.sea_water.species_mass_flow,  # This is the spices mass flow
        "convert_to_rkt_species": True,  # We can use default converter as its defined for default database (Phreeqc and pitzer)
        "activity_model": reaktoro.ActivityModelPitzer(),  # Can provide a string, or Reaktoro initialized class
        "fixed_solvent_specie": "H2O",  # We need to define our aqueous solvent as we have to speciate the block
    },
    outputs=m.fs.sea_water.outputs,  # outputs we desired    
    database_file="pitzer.dat",  # needs to be a string that names the database file or points to its location
    dissolve_species_in_reaktoro=True,  # This will sum up all species into elements in Reaktoro directly, if set to false, it will build Pyomo constraints instead
    assert_charge_neutrality=False,  # This is True by Default, but here we actually want to adjust the input speciation till the charge is zero
)

2024-09-08 15:10:18 [INFO] idaes.reaktoro_pse.core.reaktoro_inputs: Exact speciation is not provided! Fixing aqueous solvent and, excluding H
2024-09-08 15:10:18 [INFO] idaes.reaktoro_pse.core.reaktoro_inputs: Exact speciation is not provided! Fixing aqueous solvent and, excluding O
2024-09-08 15:10:19 [INFO] idaes.reaktoro_pse.core.reaktoro_gray_box: RKT gray box using BFGS hessian type


### Inspecting Reaktoro block outputs and inputs

The gray box is constructed on reaktoro_model block

In [114]:
m.fs.sea_water.eq_reaktoro_properties.reaktoro_model.inputs.display()

inputs : Size=11, Index=fs.sea_water.eq_reaktoro_properties.reaktoro_model._input_names_set
    Key         : Lower : Value : Upper : Fixed : Stale : Domain
          CO3-2 :     0 :     1 :  None : False : False :  Reals
           Ca+2 :     0 :     1 :  None : False : False :  Reals
            Cl- :     0 :     1 :  None : False : False :  Reals
            H2O :     0 :     1 :  None : False : False :  Reals
             K+ :     0 :     1 :  None : False : False :  Reals
           Mg+2 :     0 :     1 :  None : False : False :  Reals
            Na+ :     0 :     1 :  None : False : False :  Reals
          SO4-2 :     0 :     1 :  None : False : False :  Reals
             pH :     0 :     1 :  None : False : False :  Reals
       pressure :     0 :     1 :  None : False : False :  Reals
    temperature :     0 :     1 :  None : False : False :  Reals


This inputs are converted to element sums, with in reaktoro and we can inspect them for each block
using lower level api calls 

In [115]:
""" for speciation block"""

conversion_dict = (
    m.fs.sea_water.eq_reaktoro_properties.rkt_inputs.constraint_dict
)
for element, species in conversion_dict.items():
    print(element, species)

C [(1.0, 'CO3-2')]
Na [(1.0, 'Na+')]
Mg [(1.0, 'Mg+2')]
S [(1.0, 'SO4-2')]
Cl [(1.0, 'Cl-')]
K [(1.0, 'K+')]
Ca [(1.0, 'Ca+2')]


Lets inspect outputs from reaktoro graybox model

In [116]:
m.fs.sea_water.eq_reaktoro_properties.reaktoro_model.outputs.display()

outputs : Size=20, Index=fs.sea_water.eq_reaktoro_properties.reaktoro_model._output_names_set
    Key                              : Lower : Value : Upper : Fixed : Stale : Domain
                    ('charge', None) :  None :   0.1 :  None : False : False :  Reals
                   ('density', None) :  None :   0.1 :  None : False : False :  Reals
        ('speciesActivityLn', 'H2O') :  None :   0.1 :  None : False : False :  Reals
            ('speciesAmount', 'CO2') :  None :   0.1 :  None : False : False :  Reals
          ('speciesAmount', 'CO3-2') :  None :   0.1 :  None : False : False :  Reals
           ('speciesAmount', 'Ca+2') :  None :   0.1 :  None : False : False :  Reals
            ('speciesAmount', 'Cl-') :  None :   0.1 :  None : False : False :  Reals
             ('speciesAmount', 'H+') :  None :   0.1 :  None : False : False :  Reals
            ('speciesAmount', 'H2O') :  None :   0.1 :  None : False : False :  Reals
          ('speciesAmount', 'HCO3-') :  None :

Lets inspect outputs from main reaktoro model

Note how we are missing osmoticPressure, and instead have speciesActivityLn and speciesStandardVolume as our outputs, this is becouse we osmoticPressure is a pyomo property, rather a native property supplied by reaktoro. You can inspect how this property is created by checking the:
* osmoticPressure in PyomoProperties class located in reaktoro_pse.core.reaktoro_outputs
* build_osmotic_constraint in reaktoro_pse.core.pyomo_property_writer.property_functions

This two location will also show how we can access reaktoro database to pull out fixed parameters and create a custom pyomo property. 

### Fix and initialize our concentration constraint

In [117]:
for ion, value in sea_water_composition.items():
    """ fix concentration amount"""
    m.fs.sea_water.species_concentrations[ion].fix(value)
    set_scaling_factor(m.fs.sea_water.species_concentrations[ion], 1 / value)

""" set flow to 1 kg of water"""
m.fs.sea_water.species_mass_flow['H2O'].fix(1)

""" initialize concentration constraints """
for comp, pyoobj in m.fs.sea_water.eq_sea_water_species_mass_flow.items():
    if 'H2O' in comp:
        set_scaling_factor(
            m.fs.sea_water.species_mass_flow[ion], 1 
        )
    else:
        calculate_variable_from_constraint(m.fs.sea_water.species_mass_flow[comp], pyoobj)
        set_scaling_factor(
            m.fs.sea_water.species_mass_flow[ion], 1 / m.fs.sea_water.species_mass_flow[comp].value
        )
        constraint_scaling_transform(pyoobj, 1 / m.fs.sea_water.species_mass_flow[comp].value)


### Initialize Reaktoro block.  
Reaktoro initialization does several steps:

1) Initialize input constraints propagating them from user variables to Reaktoro graybox inputs
2) Solve the Reaktoro block to get output properties 
3) Propagate Reaktoro solution through output constraints and to output variables 
4) Scale all input and output variables and constraints using either user provided scaling factors or by inverse of their value 
5) Scale the jacobian using user provided scaling or inverse of scaling factors of the gray box outputs

This will in general provide a well scaled problem. 

In [118]:
m.fs.sea_water.eq_reaktoro_properties.initialize()

""" and these are manually defined properties and variables of interest """
print(
    "Density reaktoro",
    m.fs.sea_water.density.value,
)
print(
    "Osmotic pressure",
    m.fs.sea_water.osmotic_pressure.value,
)
print("Solution reaktoro_charge", m.fs.sea_water.charge.value)

2024-09-08 15:10:19 [INFO] idaes.reaktoro_pse.reaktoro_block: ---initializing property block for fs.sea_water.eq_reaktoro_properties----
2024-09-08 15:10:19 [INFO] idaes.reaktoro_pse.core.reaktoro_state: Equilibrated successfully


2024-09-08 15:10:19 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Initialized rkt block
Density reaktoro 1023.4759353858051
Osmotic pressure 2268715.8260405306
Solution reaktoro_charge 0.05577128070860777


Displaying Reaktoro state

In [119]:
m.fs.sea_water.eq_reaktoro_properties.display_reaktoro_state()

2024-09-08 15:10:19 [INFO] idaes.reaktoro_pse.reaktoro_block: -----Displaying information for property block ------
2024-09-08 15:10:19 [INFO] idaes.reaktoro_pse.reaktoro_block: +-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |   293.0000 |    K |
| Pressure        |     1.0000 |  bar |
| Charge:         | 5.5771e-02 |  mol |
| Element Amount: |            |      |
| :: H            | 1.1101e+02 |  mol |
| :: C            | 2.3329e-03 |  mol |
| :: O            | 5.5623e+01 |  mol |
| :: Na           | 4.5917e-01 |  mol |
| :: Mg           | 5.1926e-02 |  mol |
| :: S            | 2.7575e-02 |  mol |
| :: Cl           | 4.7950e-01 |  mol |
| :: K            | 9.7192e-03 |  mol |
| :: Ca           | 9.9803e-03 |  mol |
| Species Amount: |            |      |
| :: H+           | 3.8058e-08 |  mol |
| :: H2O          | 5.5506e+01 |  mol |
| :: CO3-2        | 1.9905e-05 |  mol |
| :: CO2          | 9.

### Lets solve the current model to:
* Find actual mass flows of species 
* Solution density
* Required Cl amount to get zero charge in solution

In [120]:
""" unfix Cl and fix charge to 0"""
m.fs.sea_water.species_concentrations["Cl"].unfix()
m.fs.sea_water.charge.fix(0)

Lets check DOFs before solve, and note that its equal to number of our reaktoro outputs

In [121]:
print("DOFs:", degrees_of_freedom(m))
outputs_main_block = len(m.fs.sea_water.eq_reaktoro_properties.reaktoro_model.outputs)
print("Number of Reaktoro outputs", outputs_main_block)
print(
    "Actual DOFs:",
    degrees_of_freedom(m) - (outputs_main_block),
)
assert degrees_of_freedom(m) - (outputs_main_block) == 0

DOFs: 20
Number of Reaktoro outputs 20
Actual DOFs: 0


In [122]:
initial_cl = m.fs.sea_water.species_concentrations["Cl"].value
cy_solver = get_solver(solver="cyipopt-watertap")

cy_solver.options["max_iter"] = 25

result = cy_solver.solve(m, tee=True)
assert_optimal_termination(result)

cyipopt-watertap: cyipopt with user variable scaling and IDAES jacobian constraint scaling
This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      310
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:       56
                     variables with only lower bounds:       11
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       56
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du

In [123]:
print(
    "Density reaktoro",
    m.fs.sea_water.density.value,
)
print(
    "Osmotic pressure",
    m.fs.sea_water.osmotic_pressure.value,
)
print(
    "Solution reaktoro_charge",
    m.fs.sea_water.charge.value,
    "intial Cl",
    initial_cl,
    "final Cl",
    m.fs.sea_water.species_concentrations["Cl"].value,
)

Density reaktoro 1023.8192166756572
Osmotic pressure 2343131.1214005514
Solution reaktoro_charge 0 intial Cl 17000 final Cl 18977.219105695793


Exact speciation  

In [124]:
m.fs.sea_water.eq_reaktoro_properties.outputs.display()

outputs : Size=15, Index={('speciesAmount', 'OH-'), ('speciesAmount', 'Ca+2'), ('speciesAmount', 'Cl-'), ('speciesAmount', 'HCO3-'), ('speciesAmount', 'Mg+2'), ('speciesAmount', 'K+'), ('speciesAmount', 'HSO4-'), ('speciesAmount', 'Na+'), ('speciesAmount', 'SO4-2'), ('speciesAmount', 'MgCO3'), ('speciesAmount', 'H+'), ('speciesAmount', 'CO3-2'), ('speciesAmount', 'CO2'), ('speciesAmount', 'MgOH+'), ('speciesAmount', 'H2O')}
    Key                        : Lower : Value                  : Upper : Fixed : Stale : Domain
      ('speciesAmount', 'CO2') :  None :  8.820292774576382e-05 :  None : False : False :  Reals
    ('speciesAmount', 'CO3-2') :  None : 2.0221381093695295e-05 :  None : False : False :  Reals
     ('speciesAmount', 'Ca+2') :  None :   0.009748120528662453 :  None : False : False :  Reals
      ('speciesAmount', 'Cl-') :  None :     0.5228168599519146 :  None : False : False :  Reals
       ('speciesAmount', 'H+') :  None :  3.735931000380949e-08 :  None : False : False

## Performing acid addition (Chemistry modification)
We need to build a a specitation reaktoro block and add chemical to it.

In [125]:
""" We will want to add a dose of acid to the feed to drop the pH to typical operating point of RO"""
m.fs.acid_dose = Var(initialize=1e-8, units=pyunits.mg / pyunits.L)
set_scaling_factor(m.fs.acid_dose,1)
m.fs.acid_mass_flow = Var(
    initialize=1e-16, units=pyunits.kg / pyunits.s
) 
set_scaling_factor(m.fs.acid_mass_flow, 1e6) # we know this is roughly right as we will be adding 1-100 ppm per L or kg of feedso th emass flow will be about 1e4
""" Convert dose to mass flow """
m.fs.eq_acid_dose = Constraint(
    expr=m.fs.acid_mass_flow
    == pyunits.convert(
        m.fs.acid_dose
        * m.fs.sea_water.species_mass_flow["H2O"]
        / m.fs.sea_water.density,
        to_units=pyunits.kg / pyunits.s,
    )
)

constraint_scaling_transform(m.fs.eq_acid_dose, 1e6)

Crete a modfied sea water block and properites of interest 

In [126]:
m.fs.modified_sea_water=Block()
m.fs.modified_sea_water.pH=Var(initialize=1)
set_scaling_factor(m.fs.modified_sea_water,1)
m.fs.modified_sea_water.scaling_tendencies=Var(['Calcite', "Gypsum"], initialize=1,)
set_scaling_factor(m.fs.modified_sea_water.scaling_tendencies,1)
m.fs.modified_sea_water.outputs={
    ('pH',None):m.fs.modified_sea_water.pH
}
for key, obj in m.fs.modified_sea_water.scaling_tendencies.items():
    m.fs.modified_sea_water.outputs[('scalingTendency',key)]=obj
for key, obj in m.fs.modified_sea_water.outputs.items():
    print(key,obj)

('pH', None) fs.modified_sea_water.pH
('scalingTendency', 'Calcite') fs.modified_sea_water.scaling_tendencies[Calcite]
('scalingTendency', 'Gypsum') fs.modified_sea_water.scaling_tendencies[Gypsum]


Build speciation reaktoro block and provide acid as an input
We can use exact speciation from our eq_sea_water_properties block output as new mol flow input 

In [127]:

m.fs.modified_sea_water.eq_reaktoro_properties = ReaktoroBlock(
    exact_speciation=True,
    aqueous_phase={
        "composition": m.fs.sea_water.eq_reaktoro_properties.outputs,  # This is the spices mass flow
        "convert_to_rkt_species": False,  # already has right naming convetion
        "activity_model": reaktoro.ActivityModelPitzer(),  # Can provide a string, or Reaktoro initialized class
    },
    system_state={
        "temperature": m.fs.sea_water.temperature, # same as our feed
        "pressure": m.fs.sea_water.pressure,
    },
    outputs=m.fs.modified_sea_water.outputs,  # outputs we desired
    chemistry_modifier={
        "HCl": m.fs.acid_mass_flow
    },  # here we define that we are adding 'HCL' as acid and its mass flow
    database="PhreeqcDatabase",  # Can provide a string, or Reaktoro initialized class reaktor.PhreeqcDatabase()
    database_file="pitzer.dat",  # needs to be a string that names the database file or points to its location
    dissolve_species_in_reaktoro=True,  # This will sum up all species into elements in Reaktoro directly, if set to false, it will build Pyomo constraints instead
    assert_charge_neutrality=False,  # This is True by Default, but here we actually want to adjust the input speciation till the charge is zero
    reaktoro_solve_options={
        "open_species_on_property_block": [
                "OH-","H+",
        ]
    },  # This option helps stabilize Reaktoro by providing redundant constraints and generally does not impact final solution.
)

2024-09-08 15:10:20 [WARNING] idaes.reaktoro_pse.core.reaktoro_inputs: Registered an empty constraint for ['OH-', 'H+'], this can lead to unexpected results depending on reaktoro configuration, please use with caution
2024-09-08 15:10:20 [WARNING] idaes.reaktoro_pse.core.reaktoro_inputs: Registered an empty constraint for ['OH-', 'H+'], this can lead to unexpected results depending on reaktoro configuration, please use with caution
2024-09-08 15:10:20 [INFO] idaes.reaktoro_pse.core.reaktoro_gray_box: RKT gray box using BFGS hessian type


Lets inspect how this block sums up true species to elements 

In [128]:
""" for speciation block"""

conversion_dict = (
    m.fs.modified_sea_water.eq_reaktoro_properties.rkt_inputs.constraint_dict
)
for element, species in conversion_dict.items():
    print(element, species)

H [(1.0, 'OH-'), (1.0, 'HCO3-'), (1.0, 'HSO4-'), (1.0, 'H+'), (1.0, 'MgOH+'), (2.0, 'H2O'), (1, 'HCl')]
C [(1.0, 'HCO3-'), (1.0, 'MgCO3'), (1.0, 'CO3-2'), (1.0, 'CO2')]
O [(1.0, 'OH-'), (3.0, 'HCO3-'), (4.0, 'HSO4-'), (4.0, 'SO4-2'), (3.0, 'MgCO3'), (3.0, 'CO3-2'), (2.0, 'CO2'), (1.0, 'MgOH+'), (1.0, 'H2O')]
Na [(1.0, 'Na+')]
Mg [(1.0, 'Mg+2'), (1.0, 'MgCO3'), (1.0, 'MgOH+')]
S [(1.0, 'HSO4-'), (1.0, 'SO4-2')]
Cl [(1.0, 'Cl-'), (1, 'HCl')]
K [(1.0, 'K+')]
Ca [(1.0, 'Ca+2')]


Initialize the spectiation block

In [129]:
m.fs.modified_sea_water.eq_reaktoro_properties.initialize()

2024-09-08 15:10:20 [INFO] idaes.reaktoro_pse.reaktoro_block: ---initializing property block for fs.modified_sea_water.eq_reaktoro_properties----


2024-09-08 15:10:20 [INFO] idaes.reaktoro_pse.core.reaktoro_state: Equilibrated successfully
2024-09-08 15:10:20 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Initialized rkt block


Verify out properties look good

Note, how the pH thats estimated by Reaktoro is the same as our sea_water feed pH, even as we did not provide as an input! This is because we did not add any acid yet.

In [130]:
print('Sea water pH', m.fs.sea_water.pH.value)
for key, obj in m.fs.modified_sea_water.outputs.items():
    print(key,obj.value)

Sea water pH 7.56
('pH', None) 7.560000000058571
('scalingTendency', 'Calcite') 0.94430589265305
('scalingTendency', 'Gypsum') 0.2001546710736435


Lets find how much acid we need to add to drop pH to 7

In [131]:
m.fs.modified_sea_water.pH.fix(7.0)

In [132]:
cy_solver = get_solver(solver="cyipopt-watertap")

cy_solver.options["max_iter"] = 25

result = cy_solver.solve(m, tee=True)

cyipopt-watertap: cyipopt with user variable scaling and IDAES jacobian constraint scaling
This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      409
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:       81
                     variables with only lower bounds:       29
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       81
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du

In [139]:
print(
    "Adjusted pH",
    m.fs.modified_sea_water.pH.value,
    "Acid dose",
    m.fs.acid_dose.value,
    "PPM, Acid mass flow",
    m.fs.acid_mass_flow.value,
    "kg/s"
)

Adjusted pH 6.5 Acid dose 25.28412956049757 PPM, Acid mass flow 2.4695892740317126e-05 kg/s


## Alternatively ReaktoroBlock can be build to perform speciation internally. 

### Getting basic properties from ReaktoroBlock
<img src="spec_block.png" width="1200" height="400">

Build combined block and output properties (same as for modified_sea_water block)

In [134]:
m.fs.combined_block=Block()
m.fs.combined_block.pH=Var(initialize=1)
set_scaling_factor(m.fs.combined_block,1)
m.fs.combined_block.scaling_tendencies=Var(['Calcite','Gypsum'], initialize=1,)
set_scaling_factor(m.fs.combined_block.scaling_tendencies,1)
m.fs.combined_block.outputs={
    ('pH',None):m.fs.combined_block.pH
}

for key, obj in m.fs.combined_block.scaling_tendencies.items():
    m.fs.combined_block.outputs[('scalingTendency',key)]=obj
for key, obj in m.fs.combined_block.outputs.items():
    print(key,obj)

('pH', None) fs.combined_block.pH
('scalingTendency', 'Calcite') fs.combined_block.scaling_tendencies[Calcite]
('scalingTendency', 'Gypsum') fs.combined_block.scaling_tendencies[Gypsum]


Build rektoro block with speciation, note how we now set build_speciation_block option to true. 
We also use the sea_water block inputs and species flows.

In [135]:

m.fs.combined_block.eq_reaktoro_properties = ReaktoroBlock(
    build_speciation_block=True,
    system_state={
        "temperature": m.fs.sea_water.temperature,
        "pressure": m.fs.sea_water.pressure,
        "pH": m.fs.sea_water.pH,
    },
    aqueous_phase={
        "composition": m.fs.sea_water.species_mass_flow,  # This is the spices mass flow
        "convert_to_rkt_species": True,  # We can use default converter as its defined for default database (Phreeqc and pitzer)
        "activity_model": reaktoro.ActivityModelPitzer(),  # Can provide a string, or Reaktoro initialized class
        "fixed_solvent_specie": "H2O",  # We need to define our aqueous solvent as we have to speciate the block
    },
    chemistry_modifier={
        "HCl": m.fs.acid_mass_flow
    },  # here we define that we are adding 'HCL' as acid and its mass flow
    outputs=m.fs.combined_block.outputs,  # outputs we desired    
    database_file="pitzer.dat",  # needs to be a string that names the database file or points to its location
    dissolve_species_in_reaktoro=True,  # This will sum up all species into elements in Reaktoro directly, if set to false, it will build Pyomo constraints instead
    assert_charge_neutrality=False,  # This is True by Default, but here we actually want to adjust the input speciation till the charge is zero
    reaktoro_solve_options={
        "open_species_on_property_block": [
                "OH-","H+",
        ]
    },  # This option helps stabilize Reaktoro by providing redundant constraints and generally does not impact final solution.

)

2024-09-08 15:10:22 [INFO] idaes.reaktoro_pse.core.reaktoro_inputs: Exact speciation is not provided! Fixing aqueous solvent and, excluding H
2024-09-08 15:10:22 [INFO] idaes.reaktoro_pse.core.reaktoro_inputs: Exact speciation is not provided! Fixing aqueous solvent and, excluding O
2024-09-08 15:10:22 [INFO] idaes.reaktoro_pse.core.reaktoro_gray_box: RKT gray box using BFGS hessian type
2024-09-08 15:10:22 [WARNING] idaes.reaktoro_pse.core.reaktoro_inputs: Registered an empty constraint for ['OH-', 'H+'], this can lead to unexpected results depending on reaktoro configuration, please use with caution
2024-09-08 15:10:22 [WARNING] idaes.reaktoro_pse.core.reaktoro_inputs: Registered an empty constraint for ['OH-', 'H+'], this can lead to unexpected results depending on reaktoro configuration, please use with caution
2024-09-08 15:10:23 [INFO] idaes.reaktoro_pse.core.reaktoro_gray_box: RKT gray box using BFGS hessian type


Initialize the block

Note how we get initialization messages for two blocks, the speciation block and property block.

In [136]:
m.fs.combined_block.eq_reaktoro_properties.initialize() 

2024-09-08 15:10:23 [INFO] idaes.reaktoro_pse.reaktoro_block: ---initializing speciation block for fs.combined_block.eq_reaktoro_properties----
2024-09-08 15:10:23 [INFO] idaes.reaktoro_pse.core.reaktoro_state: Equilibrated successfully
2024-09-08 15:10:23 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Initialized rkt block
2024-09-08 15:10:23 [INFO] idaes.reaktoro_pse.reaktoro_block: ---initializing property block for fs.combined_block.eq_reaktoro_properties----
2024-09-08 15:10:23 [INFO] idaes.reaktoro_pse.core.reaktoro_state: Equilibrated successfully
2024-09-08 15:10:23 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Initialized rkt block


Set new pH target and solve all the blocks again

In [137]:
m.fs.modified_sea_water.pH.fix(6.5)

cy_solver = get_solver(solver="cyipopt-watertap")

cy_solver.options["max_iter"] = 25

result = cy_solver.solve(m, tee=True)



cyipopt-watertap: cyipopt with user variable scaling and IDAES jacobian constraint scaling
This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      734
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       11

Total number of variables............................:      146
                     variables with only lower bounds:       58
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      146
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du

Compare outputs between modified block and combine blocks

In [138]:
for key in m.fs.combined_block.outputs:
    print(key, 'modified block result', m.fs. modified_sea_water.outputs[key].value, 'combined block result', m.fs.combined_block.outputs[key].value, )

('pH', None) modified block result 6.5 combined block result 6.499999999992061
('scalingTendency', 'Calcite') modified block result 0.059140695044277006 combined block result 0.059140695042849266
('scalingTendency', 'Gypsum') modified block result 0.20015129373414978 combined block result 0.2001512937341508
